# Introduccion a Langchain

## Antes de comenzar...

Durante a lo largo de este repositorio estare hablando acerca de lo aprendido en langchain y como estuve trabajando con el, es por ello que sera necesario correr los modelos de manera local.
En este caso estaremos haciendo el uso de Ollama y estaremos descargando algunos modelos segun se vayan necesitando.
Para consultar mas informacion acerca de Ollama presiona [aqui.](https://github.com/Sanchez1909/Introduccion-a-Langchain/blob/main/Ollama/Ollama.md)

## Componentes esenciales

Para llevar poder llevar acabo este curso, sera necesario tener instalado:
- Visual Studio Code
- Ollama
    - nomic-embed-text (Nos servira para trabajar con los embeddings)
    - llama3.2 / deepseek-r1:1.5b
- Python
- Jupyter Notebook/Google Colab

---

## Navegando por langchain

En estos momentos, estaremos navegando por diferentes componentes de langchain para poder conocer sus diversas funcionalidades y poder trabajar con la biblioteca.

---

### LLM Wrappers

Estos componentes son abstracciones que encapsulan la interaccion con modelos de lenguaje de gran escala (LLM), facilitando la integracion de estos modelos en nuestras aplicaciones y añadiendoles diversas funcionalidades.

In [8]:
!pip install langchain_ollama -q
!pip install langchain_community -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import ollama 
from langchain_ollama import ChatOllama 
from langchain_community.llms import Ollama

In [10]:
llm =  Ollama(model= "llama3.2")

respuesta = llm.invoke("Cuentame un chiste")

respuesta

'¡Claro! Aquí tienes uno:\n\n¿Por qué la computadora fue al doctor?\n\n¡Porque tenía un virus!\n\nEspero que te haya hecho sonreír. ¿Quieres escuchar otro?'

### Chat Models

Este componente es un tipo especializado de modelo de lenguaje que procesa y responde a entradas en formato de mensajes de chat, en lugar de texto plano.

In [12]:
llmChat = ChatOllama(model="llama3.2")

respuesta = llm.invoke("Cuentame un chiste")

respuesta

'¡Claro! Aquí tienes uno:\n\n¿Por qué la computadora fue al doctor?\n\nPorque tenía un virus!\n\nEspero que te haya hecho sonreír. ¿Quieres escuchar otro?'

> Importante tener en cuenta que el tiempo de procesamiento de la respuesta depende de la capacidad de nuestra computadora y del tamaño del modelo.

### Prompt Templates

Los Prompt Templates son plantillas que nos facilitan la generacion de prompts para poder unirlos con los modelos de lenguaje y conseguir resultados mas especificos.

Son muy utilices, ya que son bastante flexibles y nos permiten que la necesidad de su rediseño sea minima.

In [25]:
!pip install langchain -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from langchain.prompts import PromptTemplate

In [19]:
promptTemplate = PromptTemplate.from_template(
    "Dime algo {adjetivo} sobre {tema}"
)

promptTemplate.format(adjetivo="gracioso",tema= "los oso")

'Dime algo gracioso sobre los oso'

### Chains 

- Permiten secuenciar tareas y procesos en un flujo de trabajo definido
- Hacen que la interaccion entre un componente y una herramienta sea mas facil
- Permiten una mejor optimizacion de tareas complejas y multifáceticas. 

In [23]:
chain = promptTemplate | llm

In [24]:
chain.invoke({"adjetivo": "gracioso","tema": "los osos"})

'¡Claro! Aquí te dejo una historia graciosa sobre osos:\n\nEn un bosque lleno de árboles altos y senderos sinuados, vivía un oso llamado Ollie. Ollie era conocido por su peculiaridad: cada vez que se sentía ansioso o nervioso, solía hacer una baileta extraña con sus patas.\n\nUn día, mientras Ollie estaba recorriendo el bosque en busca de comida, se topó con un grupo de osos jóvenes que estaban discutiendo sobre quién era el mejor bailarín del bosque. Los osos jóvenes habían escuchado hablar de Ollie y su baileta y decidieron investigar.\n\nCuando los osos jóvenes se acercaron a Ollie, éste les dijo: "Sí, soy el mejor bailarín del bosque. Mi baileta es la más perfecta y mi pasión por la danza es inigualable".\n\nLos osos jóvenes se miraron entre sí con sorpresa y preguntaron: "¿Cómo haces eso? ¿Te entrenaste?"\n\nOllie respondió: "Sí, me entrené durante años en el arte de hacer una baileta sin que nadie lo note. ¡Y puedo hacerlo mientras estoy comiendo también!"\n\nLos osos jóvenes se 

### Memoria

- Mejora la capacidad del sistema para recordar interacciones previas
- Permite que las generaciones de respuestas sean mas precisas y esten contextualizadas
- Crucial para aplicaciones con interacciones a largo plazo.


In [28]:
!pip install langchain_core -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama 

model = ChatOllama(model="llama3.2")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente de IA que se especializa en {habilidad}. Trata de responder todas las preguntas que esten relacionadas sobre {habilidad}"
        ),
        MessagesPlaceholder(variable_name="historial"),
        (
            "human",
            "{entrada}"
        ),
    ]
)

chainHistory = prompt | model

In [31]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

sesiones = {}

def verificarSesionHistorial(session_id: str) -> BaseChatMessageHistory:
    if session_id not in sesiones:
        sesiones[session_id] = ChatMessageHistory()
    return sesiones[session_id]

modeloConHistorial = RunnableWithMessageHistory(
    chainHistory,
    verificarSesionHistorial,
    input_messages_key="entrada",
    history_messages_key="historial",
)

In [33]:
modeloConHistorial.invoke(
    {"habilidad" : "inteligencia artificial", "entrada" : "Que son los embbedings?"},
    config={"configurable": {"session_id": "12345"}}
)

AIMessage(content='Los embeddings, también conocidos como vectores de representación o vectores de espacio de datos, son una técnica utilizada en la inteligencia artificial y el aprendizaje automático. Fue introducida por primera vez en 2013 por el equipo de researchers liderado por Quoc V. Le, durante su investigación sobre redes neuronales con convoluciones.\n\n**¿Qué es un embedding?**\n\nUn embedding es una transformación lineal que se aplica a un vector de entrada (como un documento de texto o una imagen) para reducir su dimensión a un espacio de datos más bajo y comprensible. En otras palabras, un embedding intenta capturar la esencia o el significado de los datos originales en un nuevo espacio de datos.\n\n**Características de los embeddings**\n\nLos embeddings tienen varias características importantes:\n\n1.  **Reducción de dimensionalidad**: Los embeddings reducen la dimensión de los datos originales, lo que hace más fácil el procesamiento y el almacenamiento de grandes conjun

In [36]:
modeloConHistorial.invoke(
    {"habilidad" : "inteligencia artificial", "entrada" : "Puedes ser volver a explicarlo con menos palabras?"},
    config={"configurable": {"session_id": "12345"}}
)

AIMessage(content='¡Claro! Los embeddings son una técnica utilizada en inteligencia artificial para reducir la dimensionalidad de los datos originales y capturar su significado en un nuevo espacio de datos.\n\n**Características**\n\n1.  **Reducción de dimensionalidad**: Reduce la dimensión de los datos originales.\n2.  **Representación de significado**: Intenta capturar el significado de los datos originales.\n\n**Tipos**\n\n1.  **Vector espacial**: Representa los datos como vectores en un espacio de dimensionalidad fija.\n2.  **Eje espacial**: Busca minimizar la distancia entre dos puntos en el espacio de datos originales.\n\n**Aplicaciones**\n\n1.  **Reconocimiento de patrones**\n2.  **Clasificación**\n3.  **Comprensión del lenguaje natural**', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-13T03:04:21.0086859Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22040026500, 'load_duration': 20018200, 'prompt_eval_count': 627, 'prompt_eval_d

In [37]:
modeloConHistorial.invoke(
    {"habilidad" : "inteligencia artificial", "entrada" : "Puedes ser volver a explicarlo con menos palabras?"},
    config={"configurable": {"session_id": "123ab"}}
)

AIMessage(content='¡Claro! Me especializo en inteligencia artificial y estaré aquí para ayudarte con cualquier pregunta o tema relacionado con ella. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-13T03:04:32.3665672Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3694615900, 'load_duration': 19060100, 'prompt_eval_count': 71, 'prompt_eval_duration': 352538000, 'eval_count': 37, 'eval_duration': 3320732000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-87ba896c-ee42-4653-a4ab-4102f86ef704-0', usage_metadata={'input_tokens': 71, 'output_tokens': 37, 'total_tokens': 108})

In [38]:
print(sesiones)

{'12345': InMemoryChatMessageHistory(messages=[HumanMessage(content='Que son los embbedings?', additional_kwargs={}, response_metadata={}), AIMessage(content='Los embeddings, también conocidos como vectores de representación o vectores de espacio de datos, son una técnica utilizada en la inteligencia artificial y el aprendizaje automático. Fue introducida por primera vez en 2013 por el equipo de researchers liderado por Quoc V. Le, durante su investigación sobre redes neuronales con convoluciones.\n\n**¿Qué es un embedding?**\n\nUn embedding es una transformación lineal que se aplica a un vector de entrada (como un documento de texto o una imagen) para reducir su dimensión a un espacio de datos más bajo y comprensible. En otras palabras, un embedding intenta capturar la esencia o el significado de los datos originales en un nuevo espacio de datos.\n\n**Características de los embeddings**\n\nLos embeddings tienen varias características importantes:\n\n1.  **Reducción de dimensionalidad*

### Herramientas (Tools)

- Le permite a los modelos de lenguaje extender sus capacidades con funciones especificas.
- Incluyen herramientas para busqueda, analasis y mas.
- Integra APIs y servicioes externos para agregar funcionalidades.

In [42]:
pip install wikipedia -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [51]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=150)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [45]:
tool.name

'wikipedia'

In [46]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [47]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [52]:
tool.run("Langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As'

In [66]:
!pip install -q duckduckgo-search 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

In [68]:
search.invoke("Obama")

c:\Users\edgar\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'snippet: Barack Obama has the highest favorability rating among all living presidents and Joe Biden has the lowest, according to a new Gallup poll released Tuesday. The results of a Gallup survey conducted ..., title: Obama has highest favorability rating of living presidents - POLITICO, link: https://www.politico.com/news/2025/02/11/barack-obama-joe-biden-favorability-00203578, snippet: Obama, in his remarks, insisted that he is "convinced that if we want democracy as we understand it to survive," people must work for a renewed dedication to pluralist principles., title: Obama, in 1st remarks since election, says \'a line has been ... - Fox News, link: https://www.foxnews.com/politics/obama-first-remarks-since-election-says-a-line-has-been-crossed-one-side-makes-certain-moves, snippet: Obama\'s personal charisma, stirring oratory, and his campaign promise to bring change to the established political system resonated with many Democrats, especially young and minority voters. On Januar

> El uso de las herramientas nos permiten agregar diversas funcionalidades a nuestros programas, permitiendo que estos sean un poco mas concientes fuera de sus datos de entramiento. El uso y las herramientas que uno utilice, siempre dependeran del objetivo de nuestro programa.
>
> Es necesario tener en mente que aunque podemos agregar diversas funcionalidades y langchain es gratis, algunas herramientas no lo son, por lo que es necesario conocer la herramienta que deseamos utilizar, para asi, saber si son de paga o de libre acceso.
>
> Puedes consultar todas las herramientas de langchain pulsando [aqui](https://python.langchain.com/docs/integrations/tools/)